In [1]:
#!/usr/bin/env python
import numpy as np
import math as m
import pylab as pl
from matplotlib import colors
from matplotlib import gridspec
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter
from colormaps import spectral
from matplotlib.colors import ColorConverter

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# Font properties
# Note: TNR required for bolding due to font manager bug
# Math font for compact, times-like scientific notation
# Exit out of notebook before changing
times_font=True
if times_font:
    font_bold = fm.FontProperties(family='Times New Roman', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Times', style='normal', size=14, weight='normal', stretch='normal')
    font_name ='Times'
    plt.rcParams["mathtext.fontset"] = "stix"#"stixsans"
else:
    font_name ='Dejavu Sans'
    font_bold = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='normal', stretch='normal')


In [14]:
# Stamp size power figure
# (Figure 4)

# Spectral custom colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)
ColorConverter.colors['k']=(37/256, 42/256, 70/256)

# Data file names
data_dir   = '/Users/nicholebarry/MWA/data/beam_kernel_stamps/'
data_base  = '1061316296_gridded_uvf__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
stamp_size = ['1e4','1e5','1e6','1e7','1e8']
stamp_style= ['control','half','fwhm']

# Plot styles per data set
ls=['solid','dotted','dashed']
colors=['j','i','h','g','e']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

data = np.loadtxt('/Users/nicholebarry/MWA/data/theory_z7_power_1d.txt')
k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
n_k_edges = len(k_edges)
k_edges = k_edges / .704
k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
delta = (power)*(k_center**3.)/(2*np.pi**2)
k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
delta_hist  = np.repeat(delta,2) 
#pl.plot(k_edges_hist, delta_hist, ls='dashed', color='k',linewidth=2)
pl.fill_between(k_edges_hist,delta_hist,alpha=.1,color='k')


for style_i, style_name in enumerate(stamp_style):
    for size_i, size_name in enumerate(stamp_size):
        #
        # Load data and select
        data = np.loadtxt(data_dir+data_base+style_name+'_'+size_name+'eq.txt')
        k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
        power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
        # Construct k center from the edges
        n_k_edges = len(k_edges)
        k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
        # Construct power in mK^2 units
        delta = (power)*(k_center**3.)/(2*np.pi**2)
        
        # Create histogram style data set by repeating the data 
        # Duplicate the bin edges so each bin has left and right data point
        # Duplicate the power so each bin edge has a measurement
        k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
        delta_hist  = np.repeat(delta,2) 
        
        # Plot the data on the initialised figure
        pl.plot(k_edges_hist, delta_hist, ls=ls[style_i], color=colors[size_i],linewidth=2)

        
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(2e-2,2)
pl.ylim(1e-8, 1e8)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
pl.yticks([1e-5,1e0,1e5],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Complicated double legend
f1_label = ['0.01%','0.001%','0.0001%','0.00001%','0.000001%']
f2_label = ['Gaussian decomposition','Half-power sidelobes','Matched FWHM']
# Legend 1
patches = [ pl.plot([],[], ls='solid', mec=None, color=colors[size_i], 
            label="{:s}".format(f1_label[size_i]) )[0]  for size_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.175, 0.47),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
# Legend 2
patches = [ pl.plot([],[], ls=ls[style_i], mec=None, color='grey', 
            label="{:s}".format(f2_label[style_i]) )[0]  for style_i in range(len(f2_label)) ]
legend2 = pl.legend(handles=patches, bbox_to_anchor=(0.235, 0.14), loc='center',ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend2.set_title(title="Kernel shape", prop=font_bold)
legend2._legend_box.align='left'

a.add_artist(legend1)
a.add_artist(legend2)

pl.savefig(data_dir+'Power_Spectrum_Plot.pdf', bbox_inches='tight', dpi=128)
pl.close('all')


In [13]:
# Other errors power figure
# (Figure 1)

# Spectral custom colors
ColorConverter.colors['k']=(37/256, 42/256, 70/256)
#ColorConverter.colors['g']=(87/256, 87/256, 165/256)
#ColorConverter.colors['h']=(73/256, 161/256, 178/256)
#ColorConverter.colors['i']=(241/256, 234/256, 145/256)
#ColorConverter.colors['j']=(228/256, 61/256, 78/256)
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)


# Data file names
data_dir   = '/Users/nicholebarry/MWA/data/beam_kernel_base_errors/'
data_base  = ['1061316296_gridded_uvf__even_odd_joint_noimgclip_','_averemove_','_dencorr_kperplambda10-50_1dkpower_']
error_style= ['input','float','res50','input','theory']
error_style= ['input','res50','float','input','input','theory']
type_inc   = ['dirty_xx_dft','model_xx_dft','dirty_xx_dft','dirty_xx_dft','res_xx_dft']
spec_win   = ['swbh','swbh7','swbh7','swbh7','swbh7']

# Plot styles per data set
ls=['solid','solid','solid','solid','solid','dashed']
colors=['j','i','h','g','e','k']
lg_colors=['e','g','h','i','j','k']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

for error_i, error_name in enumerate(error_style):
    #
    # Load data and select
    if error_name == 'theory':
        data = np.loadtxt('/Users/nicholebarry/MWA/data/theory_z7_power_1d.txt')
    else:
        data = np.loadtxt(data_dir+data_base[0]+type_inc[error_i]+
                          data_base[1]+spec_win[error_i]+data_base[2]+error_name+'.txt')
    k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = abs(power)*(k_center**3.)/(2*np.pi**2)
    #theory is in Mpc^-1
    if error_name == 'theory':
        k_edges = k_edges / .704
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
    delta_hist  = np.repeat(delta,2) 
        
    # Plot the data on the initialised figure
    pl.plot(k_edges_hist, delta_hist, ls=ls[error_i], color=colors[error_i],linewidth=2)
    if error_name == 'theory':
        pl.fill_between(k_edges_hist,delta_hist,alpha=.1,color=colors[error_i])
        
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(2e-2,2)
pl.ylim(1e-8, 1e8)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
pl.yticks([1e-5,1e0,1e5],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
f1_label = ['4-term BH spectral window','5x smaller kernel resolution','Floating point precision',
            'Control simulation','Control with 70% flux subtracted', 'EoR z=7 theory']
patches = [ pl.plot([],[], ls=ls[type_i], mec=None, color=colors[type_i], 
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.29, 0.21),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
a.add_artist(legend1)

pl.savefig(data_dir+'Power_Spectrum_Plot.pdf', bbox_inches='tight', dpi=128)
pl.close('all')

In [8]:
# Style (no clip!), stamp size (clip!), res upgrade (no clip!) cal power figure
# (Figure x)

# Colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

# Data file names
data_dir   = '/Users/nicholebarry/MWA/data/beam_kernel_cal/'
data_base  = '1061316296_gridded_uvf__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
stamp_size = ['1e9eq','1e9eq','1e9eq','1e4eq','1e9eq_res500']#,'1e5','1e6','1e7','1e8']
bias_corr = [1.0000000420260982,(1.0098587411856537),1.0000001056316601,1,1]
             #(1.0011249974692782),( 1.0001024052544933),(1.0000134070335154),(1.0000014233046655)]
stamp_style= ['control','half','fwhm','control','control']

# Plot styles per data set
#ls=['solid','dotted','dashed','dashed','dashed']
ls=['solid','solid','solid','solid','dashed']
ls_legend=['solid','solid','solid','dashed','solid']
#ls_legend=['dashed','dotted','solid','solid','solid']
colors=['e','i','j','h','g']
lg_colors=['j','i','h','g','e']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
#pl.figure(figsize=(8,6))
ff = pl.figure(figsize=(8,6))
gs = gridspec.GridSpec(2, 1, height_ratios=[3,1] )
ax0 = pl.subplot(gs[0])

for iter_i, size_name in enumerate(stamp_size):
    #
    # Load data and select
    data = np.loadtxt(data_dir+data_base+stamp_style[iter_i]+'_'+size_name+'.txt')
    k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = power*(k_center**3.)/(2*np.pi**2)
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
    delta_hist  = np.repeat(delta,2) 
    if (iter_i == 0):
            delta_hist_1e9 = delta_hist
    else:
            delta_hist = delta_hist*bias_corr[iter_i] #- delta_hist_1e9*bias_corr[0])
    delta_hist = abs(delta_hist / delta_hist_1e9 - 1)
    print(np.nanmax(delta_hist))
    print(np.nanmin(delta_hist))
        #if (style_i == 0) and (size_i > 0):
        #    delta_hist = abs(delta_hist - 1)
        
        # Plot the data on the initialised figure
    ax0.plot(k_edges_hist, delta_hist, ls=ls[iter_i], color=colors[iter_i],linewidth=2)
        
# Final plotting settings and save

ax0.loglog()
ax0.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
ax0.set_ylim(1e-5, 1e5)
ax0.set_ylabel(r'', fontsize=fontsize, family=font_name)
ax0.set_yticks([1e-3,1e0,1e3])
ax0.set_yticklabels([r'10$^{-3}$','10$^{0}$','10$^{3}$'],fontsize=fontsize, family=font_name)
ax0.set_xlim(2e-2,2)
ff.text(0.028, .48, 'Percent difference',rotation='vertical',fontsize=fontsize,fontproperties=font)
#ff.text(0.042, .55, r'$\Delta^2$ (mK$^2$)',rotation='vertical',fontsize=fontsize,fontproperties=font)

#ax0.get_xaxis().set_ticks([])

# Legend 1
f1_label = ['Matched FWHM','Half-power sidelobes','Gaussian decomposition, 0.01% cut','Gaussian decomposition, 2x higher res']
patches = [ pl.plot([],[], ls=ls_legend[style_i], mec=None, color=lg_colors[style_i], 
            label="{:s}".format(f1_label[style_i]) )[0]  for style_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.5, 0.905),loc='center', ncol=2, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font,columnspacing=1.0)
legend1._legend_box.align='left'

# Tick formatting




a1=pl.gca()
a1.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a1.yaxis.set_minor_formatter(ticker.NullFormatter())


# Initialise your plot
ax1 = plt.subplot(gs[1])
ax1.plot(k_edges_hist, delta_hist_1e9, ls=ls[0], color=colors[0],linewidth=2)
ax1.loglog()
ax1.set_xlim(2e-2,2)
ax1.set_ylim(1e2,1e5)
ax1.set_yticks([1e3,1e4])
ax1.set_yticklabels([r'10$^{3}$','10$^{4}$'],fontsize=fontsize, family=font_name)
ax1.set_ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
ax1.set_xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)
ax1.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True)
ax1.set_xticks([1e-1,1e0])
ax1.set_xticklabels([r'10$^{-1}$','10$^{0}$'],fontsize=fontsize, family=font_name)
ff.text(0.015, .15, 'Reference',rotation='vertical',fontsize=fontsize,fontproperties=font)

#pl.xticks(fontname = font_name)
#pl.yticks(fontname = font_name)
pl.subplots_adjust(hspace=0.001)

f1_label2 = ['Gaussian decomposition']
patches2 = [ pl.plot([],[], ls='solid', mec=None, color=colors[0], 
            label="{:s}".format(f1_label2[size_i]) )[0]  for size_i in range(len(f1_label2)) ]
legend2 = pl.legend(handles=patches2, bbox_to_anchor=(0.22, 0.15),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend2._legend_box.align='left'

#a.add_artist(legend1)

pl.savefig(data_dir+'Power_Spectrum_Plot.pdf', bbox_inches='tight', dpi=128)
pl.close('all')

/var/folders/bq/wmsr1m_d2c94qwkmz9xyr3g80000gn/T/ipykernel_2121/2748804898.py:60: RuntimeWarning: invalid value encountered in true_divide
  delta_hist = abs(delta_hist / delta_hist_1e9 - 1)
/var/folders/bq/wmsr1m_d2c94qwkmz9xyr3g80000gn/T/ipykernel_2121/2748804898.py:60: RuntimeWarning: invalid value encountered in true_divide
  delta_hist = abs(delta_hist / delta_hist_1e9 - 1)
/var/folders/bq/wmsr1m_d2c94qwkmz9xyr3g80000gn/T/ipykernel_2121/2748804898.py:60: RuntimeWarning: invalid value encountered in true_divide
  delta_hist = abs(delta_hist / delta_hist_1e9 - 1)
/var/folders/bq/wmsr1m_d2c94qwkmz9xyr3g80000gn/T/ipykernel_2121/2748804898.py:60: RuntimeWarning: invalid value encountered in true_divide
  delta_hist = abs(delta_hist / delta_hist_1e9 - 1)
/var/folders/bq/wmsr1m_d2c94qwkmz9xyr3g80000gn/T/ipykernel_2121/2748804898.py:60: RuntimeWarning: invalid value encountered in true_divide
  delta_hist = abs(delta_hist / delta_hist_1e9 - 1)


0.0
0.0
138.7647222097268
3.702319702675868
577.0479818625137
12.385573690512315
0.19469590669022518
8.192762772085693e-05
0.07486079214773322
0.0003614888159676477


In [2]:
# FFT vs convolve
# beam int fixed to analytic

# Spectral custom colors
ColorConverter.colors['e']=(102/256, 194/256, 165/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(94/256, 79/256, 162/256)
ColorConverter.colors['i']=(72/256, 107/256, 175/256)
ColorConverter.colors['j']=(250/256, 160/256, 91/256)

#theory
ColorConverter.colors['k']=(37/256, 42/256, 70/256)

# Data file names
data_dir   = '/Users/nicholebarry/MWA/data/beam_kernel_fft/stable_beamint/'
data_base  = '1061316296_gridded_uvf__even_odd_joint_noimgclip_model_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
#stamp_style= ['control','half','fwhm']
stamp_style= ['control']#,'half','fwhm']
#stamp_size = ['fft_dim60','fft_dim600','fft_dim1200','']
stamp_size = ['']#,'jacfft_dim60_try2','jac']


# Plot styles per data set
colors=['j','i','g','h']
#colors=['i','k']
ls=['solid','dotted','dashed']
#ls=['solid','dashed']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

data = np.loadtxt('/Users/nicholebarry/MWA/data/theory_z7_power_1d.txt')
k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
n_k_edges = len(k_edges)
k_edges = k_edges / .704
k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
delta = (power)*(k_center**3.)/(2*np.pi**2)
k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
delta_hist  = np.repeat(delta,2) 
#pl.plot(k_edges_hist, delta_hist, ls='dashed', color='k',linewidth=2)
pl.fill_between(k_edges_hist,delta_hist,alpha=.1,color='k')

for style_i, style_name in enumerate(stamp_style):
    for size_i, size_name in enumerate(stamp_size):
        #
        # Load data and select
        if style_name == 'theory':
            data = np.loadtxt('/Users/nicholebarry/MWA/data/theory_z7_power_1d.txt')
        else:
            try:
                data = np.loadtxt(data_dir+data_base+style_name+size_name+'.txt')
            except:
                continue
        k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
        power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
        # Construct k center from the edges
        n_k_edges = len(k_edges)
        k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
        # Construct power in mK^2 units
        delta = (power)*(k_center**3.)/(2*np.pi**2)
        if style_name == 'theory':
            k_edges = k_edges / .704
        
        # Create histogram style data set by repeating the data 
        # Duplicate the bin edges so each bin has left and right data point
        # Duplicate the power so each bin edge has a measurement
        k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
        delta_hist  = np.repeat(delta,2) 
        
        # Plot the data on the initialised figure
        pl.plot(k_edges_hist, delta_hist, ls=ls[style_i], color=colors[size_i],linewidth=2)
        #pl.plot(k_edges_hist, delta_hist, ls=ls[style_i], color=colors[style_i],linewidth=2)
        
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(2e-2,2)
pl.ylim(1e-8, 1e8)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
pl.yticks([1e-5,1e0,1e5],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Complicated double legend
#f1_label = ['FFT','Analytic','FFT with Jacobian','Analytic with Jacobian']
f1_label = ['FFT','Analytic']
f2_label = ['Gaussian decomposition, horizon','Half-power sidelobes, horizon','Matched FWHM, horizon']
# Legend 1
patches = [ pl.plot([],[], ls='solid', mec=None, color=colors[style_i], 
            label="{:s}".format(f1_label[style_i]) )[0]  for style_i in range(len(f1_label)) ]
#legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.218, 0.42),loc='center', ncol=1, frameon=False,
#                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.131, 0.38),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend1.set_title(title="Transform", prop=font_bold)
legend1._legend_box.align='left'
# Legend 2
patches = [ pl.plot([],[], ls=ls[style_i], mec=None, color='grey', 
            label="{:s}".format(f2_label[style_i]) )[0]  for style_i in range(len(f2_label)) ]
legend2 = pl.legend(handles=patches, bbox_to_anchor=(0.285, 0.14), loc='center',ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend2.set_title(title="Kernel shape", prop=font_bold)
legend2._legend_box.align='left'

a.add_artist(legend1)
a.add_artist(legend2)


pl.savefig(data_dir+'Power_Spectrum_Plot_step1.pdf', bbox_inches='tight', dpi=128)
pl.close('all')

In [4]:
# New cal power figure with analytic simulations
# (Figure x)

# Colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

#theory
ColorConverter.colors['k']=(37/256, 42/256, 70/256)

# Data file names
data_dir   = '/Users/nicholebarry/MWA/data/beam_kernel_cal/analytic_tests/'
data_base  = '1061316296_gridded_uvf__even_odd_joint_fullimg_dirty_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
#stamp_size = ['1e9eq','1e9eq','1e9eq','1e4eq','1e9eq_res500']#,'1e5','1e6','1e7','1e8']
#bias_corr = [1.0000000420260982,(1.0098587411856537),1.0000001056316601,1,1]
             #(1.0011249974692782),( 1.0001024052544933),(1.0000134070335154),(1.0000014233046655)]
stamp_style= ['analytic_control','analytic_half','analytic_fwhm','discrete_control','theory']


# Plot styles per data set
#ls=['solid','dotted','dashed','dashed','dashed']
ls=['solid','solid','solid','dashed','dashed']
ls_legend=['solid','solid','dashed','solid']
colors=['e','i','j','h','k','k']
lg_colors=['j','i','e','e']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))
#ff = pl.figure(figsize=(8,6))
#gs = gridspec.GridSpec(2, 1, height_ratios=[3,1] )
#ax0 = pl.subplot(gs[0])

for iter_i, stamp_name in enumerate(stamp_style):
    #
    # Load data and select
    if stamp_name == 'theory':
        data = np.loadtxt('/Users/nicholebarry/MWA/data/theory_z7_power_1d.txt')
    else:
        data = np.loadtxt(data_dir+data_base+stamp_style[iter_i]+'.txt')
    k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = power*(k_center**3.)/(2*np.pi**2)
    #theory is in Mpc^-1
    if stamp_name == 'theory':
        k_edges = k_edges / .704
        
    if (iter_i == 1):
            delta_half = delta
    if (iter_i == 2):
            delta_fwhm = delta
       
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
    delta_hist  = np.repeat(delta,2) 
    #if (iter_i == 0):
    #        delta_hist_1e9 = delta_hist
    #else:
    #        delta_hist = delta_hist*bias_corr[iter_i] #- delta_hist_1e9*bias_corr[0])
    #delta_hist = abs(delta_hist / delta_hist_1e9 - 1)
    #print(np.nanmax(delta_hist))
    #print(np.nanmin(delta_hist))
        #if (style_i == 0) and (size_i > 0):
        #    delta_hist = abs(delta_hist - 1)
        
        # Plot the data on the initialised figure
    
    if stamp_name == 'theory':
        pl.fill_between(k_edges_hist,delta_hist,alpha=.1,color=colors[iter_i])
    else:
        pl.plot(k_edges_hist, delta_hist, ls=ls[iter_i], color=colors[iter_i],linewidth=2)
        
# Final plotting settings and save
#print(k_center, abs(delta_half - delta_fwhm)/delta_half - 1)

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(2e-2,2)
pl.ylim(1e-8, 1e8)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
pl.yticks([1e-5,1e0,1e5],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Complicated double legend
#f1_label = ['FFT','Analytic','FFT with Jacobian','Analytic with Jacobian']
f1_label = ['Matched FWHM','Half-power sidelobes','Gaussian decomposition']
# Legend 1
patches = [ pl.plot([],[], ls='solid', mec=None, color=lg_colors[style_i], 
            label="{:s}".format(f1_label[style_i]) )[0]  for style_i in range(len(f1_label)) ]
#legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.218, 0.42),loc='center', ncol=1, frameon=False,
#                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.23, 0.30),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend1.set_title(title="Semi-analytic", prop=font_bold)
legend1._legend_box.align='left'
# Legend 2
f2_label = ['Gaussian decomposition']
patches = [ pl.plot([],[], ls='dashed', mec=None, color='h', 
            label="{:s}".format(f2_label[style_i]) )[0]  for style_i in range(len(f2_label)) ]
legend2 = pl.legend(handles=patches, bbox_to_anchor=(0.235, 0.095), loc='center',ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend2.set_title(title="Discrete", prop=font_bold)
legend2._legend_box.align='left'

a.add_artist(legend1)
a.add_artist(legend2)


pl.savefig(data_dir+'Power_Spectrum_Plot.pdf', bbox_inches='tight', dpi=128)
pl.close('all')

In [13]:
# Stamp size with noise power figure
# (Figure x)

# Spectral custom colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)
ColorConverter.colors['k']=(37/256, 42/256, 70/256)

# Data file names
data_dir   = '/Users/nicholebarry/MWA/data/beam_kernel_stamps/'
#data_base  = '1061316296_gridded_uvf__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
stamp_size = ['1e4eq','1e6eq','1e8eq']#,'1e8eq_noclip_noise']
stamp_style= ['control','half','fwhm']

# Plot styles per data set
ls=['solid','dotted','dashed']
colors=['j','h','e','g','i']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

data = np.loadtxt('/Users/nicholebarry/MWA/data/theory_z7_power_1d.txt')
k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
n_k_edges = len(k_edges)
k_edges = k_edges / .704
k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
delta = (power)*(k_center**3.)/(2*np.pi**2)
k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
delta_hist  = np.repeat(delta,2) 
#pl.plot(k_edges_hist, delta_hist, ls='dashed', color='k',linewidth=2)
pl.fill_between(k_edges_hist,delta_hist,alpha=.1,color='k')

#noise
data_base  = '1061316296_gridded_uvf__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
for style_i, style_name in enumerate(stamp_style):
    data = np.loadtxt(data_dir+data_base+style_name+'_1e8eq_noclip_noise.txt')
    k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
    noise = data[1:,3] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder  
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = (noise)*(k_center**3.)/(2*np.pi**2)
    
    #Scale by number of observations
    n_obs = 10000
    delta = delta / (n_obs)**2
    if style_i == 0:
        delta_control = delta
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
    delta_hist  = np.repeat(delta,2) 
        
    # Plot the data on the initialised figure
    pl.plot(k_edges_hist, delta_hist, ls=ls[style_i], color='i',linewidth=2, alpha=.5)

    
    # Data file names
data_base  = '1061316296_gridded_uvf__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh7_dencorr_kperplambda10-50_1dkpower_'
for style_i, style_name in enumerate(stamp_style):
    for size_i, size_name in enumerate(stamp_size):
        #
        # Load data and select
        data = np.loadtxt(data_dir+data_base+style_name+'_'+size_name+'.txt')
        k_edges = data[:,0] # Lower and upper edge of bin, (h Mpc^-1)
        power = data[1:,1] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
        # Construct k center from the edges
        n_k_edges = len(k_edges)
        k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
        # Construct power in mK^2 units
        delta = abs(power)*(k_center**3.)/(2*np.pi**2)
        
        # Create histogram style data set by repeating the data 
        # Duplicate the bin edges so each bin has left and right data point
        # Duplicate the power so each bin edge has a measurement
        k_edges_hist  = np.repeat(k_edges,2)[1:-1] 
        delta_hist  = np.repeat(delta,2) 
        
        # Plot the data on the initialised figure
        pl.plot(k_edges_hist, delta_hist, ls=ls[style_i], color=colors[size_i],linewidth=2)


        
# Final plotting settings and save
ls=['solid','dotted','dashed']

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(2e-2,2)
pl.ylim(1e-8, 1e8)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
pl.yticks([1e-5,1e0,1e5],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Complicated double legend
#f1_label = ['0.01%','0.001%','0.0001%','0.00001%','0.000001%']
f1_label = ['0.01%','0.0001%','0.000001%']
f2_label = ['Gaussian decomposition','Half-power sidelobes','Matched FWHM']
# Legend 1
patches = [ pl.plot([],[], ls='solid', mec=None, color=colors[size_i], 
            label="{:s}".format(f1_label[size_i]) )[0]  for size_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.175, 0.4),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
# Legend 2
patches = [ pl.plot([],[], ls=ls[style_i], mec=None, color='grey', 
            label="{:s}".format(f2_label[style_i]) )[0]  for style_i in range(len(f2_label)) ]
legend2 = pl.legend(handles=patches, bbox_to_anchor=(0.235, 0.14), loc='center',ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
legend2.set_title(title="Kernel shape", prop=font_bold)
legend2._legend_box.align='left'
# Noise legend
f3_label = ['>300hrs noise']
patches = [ pl.plot([],[], ls='solid', mec=None, color='i', 
            label="{:s}".format(f3_label[0]) )[0]  for size_i in range(len(f3_label)) ]
legend3 = pl.legend(handles=patches, bbox_to_anchor=(0.165, 0.575), loc='center',ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)

a.add_artist(legend1)
a.add_artist(legend2)
a.add_artist(legend3)

pl.savefig(data_dir+'Power_Spectrum_Plot_noise_step5.pdf', bbox_inches='tight', dpi=128)
pl.close('all')
